In [9]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import GRU,Dense,Activation

from keras.optimizers import RMSprop

from keras.callbacks import ModelCheckpoint

In [2]:
poem = """
Buffalo Bill’s
defunct
who used to
ride a watersmooth-silver
stallion
and break one two three four five pigeons just like that
Jesus

he was a handsome man
and what i want to know is
how do you like your blueeyed boy
Mister Death

Had I the heaven’s embroidered cloths,
Enwrought with golden and silver light,
The blue and the dim and the dark cloths
Of night and light and the half-light,
I would spread the cloths under your feet:
But I, being poor, have only my dreams;
I have spread my dreams under your feet;
Tread softly because you tread on my dreams.

He clasps the crag with crooked hands;
Close to the sun in lonely lands,
Ring’d with the azure world, he stands.

The wrinkled sea beneath him crawls;
He watches from his mountain walls,
And like a thunderbolt he falls.

Some say the world will end in fire,
Some say in ice.
From what I’ve tasted of desire
I hold with those who favor fire.
But if it had to perish twice,
I think I know enough of hate
To say that for destruction ice
Is also great
And would suffice.

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth;

Then took the other, as just as fair,
And having perhaps the better claim,
Because it was grassy and wanted wear;
Though as for that the passing there
Had worn them really about the same,

And both that morning equally lay
In leaves no step had trodden black.
Oh, I kept the first for another day!
Yet knowing how way leads on to way,
I doubted if I should ever come back.

I shall be telling this with a sigh
Somewhere ages and ages hence:
Two roads diverged in a wood, and I—
I took the one less traveled by,
And that has made all the difference.
"""

In [3]:
vocab = list(sorted(set(poem)))

In [4]:
len(vocab)

50

In [5]:
char_to_index = dict((c,i) for i,c in enumerate(vocab))
index_to_char = dict((i,c) for i,c in enumerate(vocab))

In [6]:
max_length = 100
steps = 5

sentences = []
next_char = []

for i in range(0,len(poem)-max_length+1,steps):
  sentences.append(poem[i:i+max_length])
  next_char.append(poem[i+max_length])

for i,j in zip(sentences,next_char):
  print(f"Sentence: {i} Next Char: {j}")
  break

Sentence: 
Buffalo Bill’s
defunct
who used to
ride a watersmooth-silver
stallion
and break one two three four  Next Char: f


In [7]:
x = np.zeros((len(sentences),max_length,len(vocab)),dtype=np.bool)
y = np.zeros((len(sentences),len(vocab)),dtype=np.bool)

<ipython-input-7-6353ec699cb4>:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences),max_length,len(vocab)),dtype=np.bool)
<ipython-input-7-6353ec699cb4>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences),len(vocab)),dtype=np.bool)


In [8]:
for i,sentence in enumerate(sentences):
  for t,char in enumerate(sentence):
    x[i,t,char_to_index[char]] = 1
  y[i,char_to_index[next_char[i]]] = 1

In [15]:
model = Sequential()
model.add(GRU(128,input_shape=(max_length,len(vocab))))
model.add(Dense(len(vocab)))
model.add(Activation('softmax'))

In [16]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 128)               69120     
                                                                 
 dense (Dense)               (None, 50)                6450      
                                                                 
 activation (Activation)     (None, 50)                0         
                                                                 
Total params: 75570 (295.20 KB)
Trainable params: 75570 (295.20 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
model_chk = ModelCheckpoint(filepath='best_model.hdf5',monitor='loss',mode='min')

In [22]:
model.compile(optimizer=RMSprop(0.01),loss='categorical_crossentropy')

In [24]:
history = model.fit(x,y,epochs=50,batch_size=128)

Epoch 1/50
3/3 [==============================] - 0s 20ms/step - loss: 2.3899
Epoch 2/50
3/3 [==============================] - 0s 18ms/step - loss: 2.2601
Epoch 3/50
3/3 [==============================] - 0s 23ms/step - loss: 2.1180
Epoch 4/50
3/3 [==============================] - 0s 18ms/step - loss: 1.9954
Epoch 5/50
3/3 [==============================] - 0s 16ms/step - loss: 1.8768
Epoch 6/50
3/3 [==============================] - 0s 19ms/step - loss: 1.7712
Epoch 7/50
3/3 [==============================] - 0s 15ms/step - loss: 1.6062
Epoch 8/50
3/3 [==============================] - 0s 14ms/step - loss: 1.5092
Epoch 9/50
3/3 [==============================] - 0s 14ms/step - loss: 1.5249
Epoch 10/50
3/3 [==============================] - 0s 16ms/step - loss: 1.3008
Epoch 11/50
3/3 [==============================] - 0s 17ms/step - loss: 1.1684
Epoch 12/50
3/3 [==============================] - 0s 21ms/step - loss: 1.0895
Epoch 13/50
3/3 [==============================] - 0s 16ms/st

In [26]:
def next_sample(probs,temp):
  probs = np.asarray(probs).astype('float64')
  probs = np.log(probs) / temp
  exp_probs = np.exp(probs)
  pred = exp_probs / np.sum(exp_probs)

  pred = np.random.multinomial(1,pred,1)
  return np.argmax(pred)

In [27]:
import random

In [28]:
def generate_text(length,diversity):

  start_index = random.randint(0,len(poem)-max_length - 1)
  generated = ""
  sentence = poem[start_index: start_index+max_length]
  generated += sentence

  for i in range(length):
    x_pred = np.zeros((1,max_length,len(vocab)))

    for t,char in enumerate(sentence):
      x_pred[0,t,char_to_index[char]] = 1

    preds = model.predict(x_pred,verbose=0)[0]
    next_index = next_sample(preds,diversity)

    next_char = index_to_char[next_index]
    generated += next_char

    sentence = sentence[1:]+next_char

  return generated

In [31]:
print(generate_text(70,0.8))

ught with golden and silver light,
The blue and the dim and the dark cloths
Of night and light and thecd assunlls
Cound thime pe thave ps ounde iro cee
Tw wav the wand th
